In [ ]:
!wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
!tar xvf "/content/genres.tar.gz?" -C "/content/"

In [ ]:
!pip install shutup
from google.colab import drive
drive.mount("/content/drive")

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from librosa import feature
from glob import glob
import pandas as pd
import numpy as np
import librosa
import csv
import scipy
import shutup
import os
import shutil
shutup.please()

Feature Extraction from Audio data and saving them in their respective .csv 📁 file 🔉

In [23]:
genre = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [24]:
def GenerateFeatures (path_to_file):
  audio = librosa.util.example_audio_file() 
  y,sr=librosa.load(path_to_file)

  y_harmonic, y_percussive = librosa.effects.hpss(y)
  tempo, beat_frames = librosa.beat.beat_track(y=y_harmonic, sr=sr)

  chroma=librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
  mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr, n_mfcc=13)
  cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  contrast=librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  zrate=librosa.feature.zero_crossing_rate(y_harmonic)

  chroma_mean=np.mean(chroma,axis=1)
  chroma_std=np.std(chroma,axis=1)
  chroma_df=pd.DataFrame()
  for i in range(0,12):
      chroma_df['chroma_mean_'+str(i)]=chroma_mean[i]
  for i in range(0,12):
      chroma_df['chroma_std_'+str(i)]=chroma_mean[i]
  chroma_df.loc[0]=np.concatenate((chroma_mean,chroma_std),axis=0)

  mfccs_mean=np.mean(mfccs,axis=1)
  mfccs_std=np.std(mfccs,axis=1)
  mfccs_df=pd.DataFrame()
  for i in range(0,13):
      mfccs_df['mfccs_mean_'+str(i)]=mfccs_mean[i]
  for i in range(0,13):
     mfccs_df['mfccs_std_'+str(i)]=mfccs_mean[i]
  mfccs_df.loc[0]=np.concatenate((mfccs_mean,mfccs_std),axis=0)

  cent_mean=np.mean(cent)
  cent_std=np.std(cent)
  cent_skew=scipy.stats.skew(cent,axis=1)[0]
  contrast_mean=np.mean(contrast,axis=1)
  contrast_std=np.std(contrast,axis=1)  

  rolloff_mean=np.mean(rolloff)
  rolloff_std=np.std(rolloff)
  rolloff_skew=scipy.stats.skew(rolloff,axis=1)[0]

  spectral_df=pd.DataFrame()
  collist=['cent_mean','cent_std','cent_skew']
  for i in range(0,7):
      collist.append('contrast_mean_'+str(i))
  for i in range(0,7):
      collist.append('contrast_std_'+str(i))
  collist=collist+['rolloff_mean','rolloff_std','rolloff_skew']
  for c in collist:
      spectral_df[c]=0
  data=np.concatenate(([cent_mean,cent_std,cent_skew],contrast_mean,contrast_std,[rolloff_mean,rolloff_std,rolloff_std]),axis=0)
  spectral_df.loc[0]=data

  zrate_mean=np.mean(zrate)
  zrate_std=np.std(zrate)
  zrate_skew=scipy.stats.skew(zrate,axis=1)[0]

  zrate_df=pd.DataFrame()
  zrate_df['zrate_mean']=0
  zrate_df['zrate_std']=0
  zrate_df['zrate_skew']=0
  zrate_df.loc[0]=[zrate_mean,zrate_std,zrate_skew]

  beat_df=pd.DataFrame()
  beat_df['tempo']=tempo
  beat_df.loc[0]=tempo

  final_df=pd.concat((chroma_df,mfccs_df,spectral_df,zrate_df,beat_df),axis=1)

  return final_df

In [25]:
GenerateFeatures('/content/genres/blues/blues.00000.au').columns

Index(['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
       'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
       'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
       'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
       'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
       'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
       'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
       'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
       'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
       'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
       'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
       'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
       'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
       'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2'

In [ ]:
dir = '/content/genres/'
columns = ['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
       'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
       'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
       'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
       'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
       'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
       'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
       'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
       'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
       'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
       'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
       'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
       'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
       'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2',
       'contrast_mean_3', 'contrast_mean_4', 'contrast_mean_5',
       'contrast_mean_6', 'contrast_std_0', 'contrast_std_1', 'contrast_std_2',
       'contrast_std_3', 'contrast_std_4', 'contrast_std_5', 'contrast_std_6',
       'rolloff_mean', 'rolloff_std', 'rolloff_skew', 'zrate_mean',
       'zrate_std', 'zrate_skew', 'tempo']

for gen in genre:
  print (f"Current gen: {gen}")
  path = dir + gen

  files = os.listdir(path+'/')
  GenreFrame = pd.DataFrame(columns=columns)

  for clip in files:
    clip_path = dir + gen + '/' + clip

    print(f'Extracting features from {clip_path}')
    clipDataFrame = GenerateFeatures(clip_path)
    clipDataFrame['label'] = gen
  
    GenreFrame = pd.concat((GenreFrame, clipDataFrame), axis=0)

  GenreFrame.to_csv('/content/' + gen + '.csv')
  print(f'Label: {gen} saved at /content/{gen}.csv\n\n\n\n')
  shutil.copy (f"/content/{gen}.csv", f"/content/drive/MyDrive/IITK DL datasets/{gen}.csv")


Reading .csv file and adding a new column type ✔

In [34]:
blues = pd.read_csv("/content/blues.csv")
classical = pd.read_csv("/content/classical.csv")
country = pd.read_csv("/content/country.csv")
disco = pd.read_csv("/content/disco.csv")
hiphop = pd.read_csv("/content/hiphop.csv")
jazz = pd.read_csv("/content/jazz.csv")
metal = pd.read_csv("/content/metal.csv")
pop = pd.read_csv("/content/pop.csv")
reggae = pd.read_csv("/content/reggae.csv")
rock = pd.read_csv("/content/rock.csv")

blues.drop(["Unnamed: 0"], inplace=True, axis=1)
classical.drop(["Unnamed: 0"], inplace=True, axis=1)
country.drop(["Unnamed: 0"], inplace=True, axis=1)
disco.drop(["Unnamed: 0"], inplace=True, axis=1)
hiphop.drop(["Unnamed: 0"], inplace=True, axis=1)
jazz.drop(["Unnamed: 0"], inplace=True, axis=1)
metal.drop(["Unnamed: 0"], inplace=True, axis=1)
pop.drop(["Unnamed: 0"], inplace=True, axis=1)
reggae.drop(["Unnamed: 0"], inplace=True, axis=1)
rock.drop(["Unnamed: 0"], inplace=True, axis=1)

blues['label'] = 'blues'
classical['label'] = 'classical'
country['label'] = 'country'
disco['label'] = 'disco'
hiphop['label'] = 'hiphop'
jazz['label'] = 'jazz'
metal['label'] = 'metal'
pop['label'] = 'pop'
reggae['label'] = 'reggae'
rock['label'] = 'rock'

In [ ]:
blues.head(10)

Creating The Main Dataframe for training our Model 🚩

In [29]:
final_df = pd.DataFrame(columns = rock.columns)

In [30]:
genres = [blues, 
          classical, 
          country, 
          disco, 
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          rock]

final_df = pd.concat(genres)

In [49]:
final_df.drop(["Unnamed: 0"], inplace=True, axis=1)

In [50]:
final_df.shape

(1000, 75)

Encoding labels of Target column ✈

In [51]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

Fitting Rest of the columns in StanderdScaler (Using fit_transform for that)

In [52]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,:-1], dtype = float))

Splitting Our Dataset for Training (x_train & y_train) and Testing (y_train & y_test)

In [53]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.33)

Creating TrainModel Function for training our RNN model.

In [54]:
def trainModel (model, epochs, optimizer, verbose):
  batch_size = 128

  model.compile(optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

  return model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 epochs=epochs, batch_size=batch_size, verbose=verbose)

Our Sequential Model ⚖

In [55]:
model = keras.models.Sequential([
                             keras.layers.Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(256, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(128, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Flatten(),
                             keras.layers.Dense(10, activation='softmax')
                          ])
print(model.summary())
model_history = trainModel(model=model, epochs=150, optimizer='adam', verbose=2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               38400     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_16 (Dense)            (None, 256)               131328    
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)               

In [59]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=128)
print (f"\nThe test loss is : {round(test_loss, 2)}")
print(f"The best test accuracy is : {round(test_acc*100, 2)}%")

3/3 [==============================] - 0s 5ms/step - loss: 2.0259 - accuracy: 0.7061

The test loss is : 2.03
The best test accuracy is : 70.61%


In [56]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               38400     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_16 (Dense)            (None, 256)               131328    
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)               

In [67]:
def makeAudio_to_Data (path):

  newDataframe = GenerateFeatures (path)

  fit2 = StandardScaler()
  numpyArray = fit2.fit_transform(np.array(newDataframe, dtype = float))

  return numpyArray, newDataframe

data, frame = makeAudio_to_Data ('/content/ambitionz az ridah.mkv')


In [68]:
arr = model.predict(data)
lis = arr.tolist()
print(genre[lis[0].index(max(lis[0]))].capitalize())

Disco


In [72]:
f30 = pd.read_csv("/content/features_30_sec.csv")

In [73]:
f30.sample()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
985,rock.00085.wav,661794,0.32172,0.088604,0.063006,0.000876,2255.709364,532079.773017,2057.933232,105722.924816,...,93.243904,-8.971213,120.844284,6.871623,134.646729,-6.688627,84.300697,-1.209258,121.966843,rock
